In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install Bio-Epidemiology-NER

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.4 MB/s eta 0:00:00


In [ ]:
import os
from Bio_Epidemiology_NER.bio_recognizer import pdf_annotate

folder_path = '/content/drive/MyDrive/MACROBATT/INPUT_PDF_FILES'  # Replace with your folder path

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):  # Check if the current item is a PDF file
        pdffile = os.path.join(folder_path, filename)

        # Annotation and export operations
        # Annotated PDF file
        #pdf_annotate(pdffile, compute='cpu', output_format='pdf')  # Pass compute='gpu' if using GPU

        # CSV file with entities
        doc_out = pdf_annotate(pdffile, compute='cpu', output_format='csv')  # Pass compute='gpu' if using GPU

        # Both annotated PDF and CSV file
        # pdf_annotate(pdffile, compute='cpu', output_format='all')  # Pass compute='gpu' if using GPU

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/Bio_Epidemiology_NER/bio_recognizer.py:94: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(disease_df) # adding the disease_df to existing
/usr/local/lib/python3.10/dist-packages/Bio_Epidemiology_NER/bio_recognizer.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_df = master_df.append(final_df)
/usr/local/lib/python3.10/dist-packages/Bio_Epidemiology_NER/bio_recognizer.py:94: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(disease_df) # adding the disease_df to existing
/usr/local/lib/python3.10/dist-packages/Bio_Epidemiology_NER/bio_recognizer.py:100: FutureWarning: The fr

In [ ]:
pip install fuzzywuzzy

In [ ]:
import os
import pandas as pd
from fuzzywuzzy import fuzz

def add_similarity_scores(df1, column1, df2, column2, threshold=70):
    df_result = df1.copy()
    df_result['Similarity Score'] = 0
    df_result['Entity Group2'] = ""
    df_result['Value2'] = ""

    true_positives = 0
    false_positives = 0
    false_negatives = 0
    total_positive_matches = 0
    total_actual_matches = 0

    df1['Actual Match Column'] = None
    df2['Actual Match Column'] = None

    for i, value1 in enumerate(df1[column1]):
        best_match_score = 0
        best_match_index = -1

        for j, value2 in enumerate(df2[column2]):
            similarity_score = fuzz.token_set_ratio(str(value1), str(value2))
            if similarity_score > best_match_score:
                best_match_score = similarity_score
                best_match_index = j

        df_result.at[i, 'Similarity Score'] = best_match_score

        if best_match_index != -1:
            df_result.at[i, 'Entity Group2'] = df2.at[best_match_index, 'Entity Group']
            df_result.at[i, 'Value2'] = df2.at[best_match_index, 'Value']

            if best_match_score >= threshold:
                total_positive_matches += 1

                if df1.at[i, 'Actual Match Column'] == df2.at[best_match_index, 'Actual Match Column']:
                    true_positives += 1
                else:
                    false_positives += 1

            if df1.at[i, 'Actual Match Column'] in df2['Actual Match Column'].values:
                total_actual_matches += 1

    false_negatives = total_actual_matches - true_positives

    precision = true_positives / total_positive_matches if total_positive_matches > 0 else 0
    recall = true_positives / total_actual_matches if total_actual_matches > 0 else 0

    return df_result, precision, recall

# Initialize variables to track cumulative statistics
total_similarity_score = 0
total_true_positives = 0
total_false_positives = 0
total_false_negatives = 0
total_positive_matches = 0
total_actual_matches = 0

# Create a list to store precision and recall for each file
precision_recall_list = []

# Define the path to the input PDF files folder
input_folder_path = '/content/drive/MyDrive/MACROBATT/INPUT_PDF_FILES/'

# Get a list of all files in the folder
input_files = os.listdir(input_folder_path)

# Iterate over each file in the folder
for file_name in input_files:
    # Check if the file is a CSV file
    if file_name.endswith('.csv'):
        # Print the file name
        print("Processing file:", file_name)

        # Construct the full file path
        input_file_path = os.path.join(input_folder_path, file_name)

        # Read the input file using pandas
        df1 = pd.read_csv(input_file_path)

        # Define the folder path for the second set of files
        annotated_folder_path = '/content/drive/MyDrive/MACROBATT/Already_annotated/'

        # Construct the file name for the second file
        file_name2 = file_name.replace('_df', '').replace('.csv', '.ann')

        # Construct the full file path for the second file
        annotated_file_path = os.path.join(annotated_folder_path, file_name2)

        try:
            # Read the annotated file using pandas, ignoring bad lines
            df2 = pd.read_csv(annotated_file_path, sep='\t', engine='python', header=None, error_bad_lines=False).drop(0, axis=1)
            df2.columns = ['Entity Group', 'Value']

            # Define the threshold for similarity score
            threshold = 70

            # Apply similarity scores and get the result DataFrame
            result_df, precision, recall = add_similarity_scores(df1, "Value", df2, "Value", threshold=threshold)

            # Accumulate statistics
            total_similarity_score += result_df['Similarity Score'].sum()
            total_true_positives += result_df['Similarity Score'].gt(70).sum()
            total_false_positives += result_df['Similarity Score'].le(70).sum()
            total_false_negatives += result_df['Similarity Score'].le(70).sum()
            total_positive_matches += result_df['Similarity Score'].gt(70).sum()
            total_actual_matches += df2['Value'].nunique()

            # Append precision and recall to the list
            precision_recall_list.append((file_name, precision, recall))

            # Print the file names
            print("File Name:", file_name)
            print("Annotated File Name:", file_name2)
            print("")

        except pd.errors.ParserError as e:
            print(f"ParserError: Skipping {annotated_file_path}, encountered an invalid line format.")
            print(e)

    else:
        print("Skipping non-CSV file:", file_name)

# Calculate overall precision and recall
overall_precision = total_true_positives / total_positive_matches if total_positive_matches > 0 else 0
overall_recall = total_true_positives / total_actual_matches if total_actual_matches > 0 else 0

# Calculate overall average similarity score
total_files = len(input_files)
overall_similarity_score_avg = total_similarity_score / total_files

# Create a DataFrame to store precision and recall values for each file
precision_recall_df = pd.DataFrame(precision_recall_list, columns=['File Name', 'Precision', 'Recall'])

# Write precision and recall DataFrame to a new CSV file
output_csv_path = '/content/drive/MyDrive/MACROBATT/Precision_Recall_Results.csv'
precision_recall_df.to_csv(output_csv_path, index=False)

# Print overall statistics
print("Overall Statistics:")
print("Total Average Similarity Score:", overall_similarity_score_avg)
print("Total True Positives:", total_true_positives)
print("Total False Positives:", total_false_positives)
print("Total False Negatives:", total_false_negatives)
print("Overall Precision:", overall_precision)
print("Overall Recall:", overall_recall)
print("")

print("Precision and Recall for each file:")
print(precision_recall_df)


Skipping non-CSV file: 28353604.pdf
Skipping non-CSV file: 28559815.pdf
Skipping non-CSV file: 28538413.pdf
Skipping non-CSV file: 28353588.pdf
Skipping non-CSV file: 28353596.pdf
Skipping non-CSV file: 28353613.pdf
Skipping non-CSV file: 28353569.pdf
Skipping non-CSV file: 28403092.pdf
Skipping non-CSV file: 28403086.pdf
Skipping non-CSV file: 28321071.pdf
Skipping non-CSV file: 28353561.pdf
Skipping non-CSV file: 28353556.pdf
Skipping non-CSV file: 28353558.pdf
Skipping non-CSV file: 28422883.pdf
Skipping non-CSV file: 28383413.pdf
Skipping non-CSV file: 28403099.pdf
Skipping non-CSV file: 28321070.pdf
Skipping non-CSV file: 28321073.pdf
Skipping non-CSV file: 28595573.pdf
Skipping non-CSV file: 28767567.pdf
Skipping non-CSV file: 28239141.pdf
Skipping non-CSV file: 28248858.pdf
Skipping non-CSV file: 28216610.pdf
Skipping non-CSV file: 28202865.pdf
Skipping non-CSV file: 28207542.pdf
Skipping non-CSV file: 28196820.pdf
Skipping non-CSV file: 28248891.pdf
Skipping non-CSV file: 28202

<ipython-input-28-1bfe3078a9ce>:95: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv(annotated_file_path, sep='\t', engine='python', header=None, error_bad_lines=False).drop(0, axis=1)


File Name: 28383413_df.csv
Annotated File Name: 28383413.ann

Processing file: 28403099_df.csv


<ipython-input-28-1bfe3078a9ce>:95: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv(annotated_file_path, sep='\t', engine='python', header=None, error_bad_lines=False).drop(0, axis=1)


File Name: 28403099_df.csv
Annotated File Name: 28403099.ann

Processing file: 28321070_df.csv


...

File Name: 28353561_df.csv
Annotated File Name: 28353561.ann

Processing file: 28353556_df.csv


<ipython-input-28-1bfe3078a9ce>:95: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv(annotated_file_path, sep='\t', engine='python', header=None, error_bad_lines=False).drop(0, axis=1)


File Name: 28353556_df.csv
Annotated File Name: 28353556.ann

Processing file: 28353558_df.csv


<ipython-input-28-1bfe3078a9ce>:95: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv(annotated_file_path, sep='\t', engine='python', header=None, error_bad_lines=False).drop(0, axis=1)


File Name: 28353558_df.csv
Annotated File Name: 28353558.ann

Overall Statistics:
Total Average Similarity Score: 5132.65
Total True Positives: 20488
Total False Positives: 206
Total False Negatives: 206
Overall Precision: 1.0
Overall Recall: 0.9311457528518838

Precision and Recall for each file:
           File Name  Precision    Recall
0    28383413_df.csv        1.0  1.000000
1    28403099_df.csv        1.0  0.953020
2    28321070_df.csv        1.0  1.000000
3    28321073_df.csv        1.0  1.000000
4    28595573_df.csv        1.0  1.000000
..               ...        ...       ...
195  28403086_df.csv        1.0  1.000000
196  28321071_df.csv        1.0  1.000000
197  28353561_df.csv        1.0  1.000000
198  28353556_df.csv        1.0  0.992537
199  28353558_df.csv        1.0  1.000000

[200 rows x 3 columns]


In [ ]:
import os
import pandas as pd

def extract_severity(df):
    # Find rows containing "severity" in the "Entity Group" column (case-insensitive and ignoring leading/trailing spaces)
    mask = df["Entity Group"].str.strip().str.lower() == "severity"

    if mask.any():
        # Get the corresponding "Value" for "severity"
        severity_value = df.loc[mask, "Value"].values[0]
        return severity_value
    else:
        return "No severity"

# Initialize a list to store results
results = []

# Define the path to the input CSV files folder
input_folder_path = '/content/drive/MyDrive/MACROBATT/INPUT_PDF_FILES/'

# Get a list of all files in the folder
input_files = os.listdir(input_folder_path)

# Iterate over each file in the folder
for file_name in input_files:
    # Check if the file is a CSV file
    if file_name.endswith('.csv'):
        # Print the file name
        print("Processing file:", file_name)

        # Construct the full file path
        input_file_path = os.path.join(input_folder_path, file_name)

        # Read the input file using pandas
        df = pd.read_csv(input_file_path)

        # Check if "severity" is present and extract its value
        severity_value = extract_severity(df)

        # Append the file name and severity value to the results list
        results.append((file_name, severity_value))

# Create a DataFrame from the results list
results_df = pd.DataFrame(results, columns=['File Name', 'Severity'])

# Define the path for the output CSV file
output_csv_path = '/content/drive/MyDrive/MACROBATT/Severity_Results.csv'

# Write the results DataFrame to the output CSV file
results_df.to_csv(output_csv_path, index=False)

# Display the results DataFrame
print("Results:")
print(results_df)


Processing file: 28383413_df.csv
Processing file: 28403099_df.csv
Processing file: 28321070_df.csv
Processing file: 28321073_df.csv
Processing file: 28595573_df.csv
Processing file: 28767567_df.csv
Processing file: 28239141_df.csv
Processing file: 28248858_df.csv
Processing file: 28216610_df.csv
Processing file: 28202865_df.csv
Processing file: 28207542_df.csv
Processing file: 28196820_df.csv
Processing file: 28248891_df.csv
Processing file: 28202862_df.csv
Processing file: 28190872_df.csv
Processing file: 28193213_df.csv
Processing file: 28202869_df.csv
Processing file: 28250304_df.csv
Processing file: 28272214_df.csv
Processing file: 28265107_df.csv
Processing file: 28250406_df.csv
Processing file: 28272235_df.csv
Processing file: 28292056_df.csv
Processing file: 28320420_df.csv
Processing file: 28296749_df.csv
Processing file: 28296775_df.csv
Processing file: 28151882_df.csv
Processing file: 27998312_df.csv
Processing file: 28121940_df.csv
Processing file: 28154281_df.csv
Processing

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import pandas as pd

def extract_severity(df):
    # Find rows containing "severity" in the "Entity Group" column (case-insensitive and ignoring leading/trailing spaces)
    mask = df["Entity Group"].str.strip().str.lower() == "severity"

    if mask.any():
        # Get the corresponding "Value" for "severity"
        severity_values = df.loc[mask, "Value"].values.tolist()
        return severity_values
    else:
        return []

# Initialize a list to store results
results = []

# Define the path to the input CSV files folder
input_folder_path = '/content/drive/MyDrive/MACROBATT/INPUT_PDF_FILES/'

# Get a list of all files in the folder
input_files = os.listdir(input_folder_path)

# Iterate over each file in the folder
for file_name in input_files:
    # Check if the file is a CSV file
    if file_name.endswith('.csv'):
        # Print the file name
        print("Processing file:", file_name)

        # Construct the full file path
        input_file_path = os.path.join(input_folder_path, file_name)

        # Read the input file using pandas
        df = pd.read_csv(input_file_path)

        # Check if "severity" is present and extract its values (could be multiple)
        severity_values = extract_severity(df)

        # Append the file name and severity values to the results list
        results.append((file_name, severity_values))

# Create a DataFrame from the results list
results_df = pd.DataFrame(results, columns=['File Name', 'Severity'])

# Define the path for the output CSV file
output_csv_path = '/content/drive/MyDrive/MACROBATT/Severity_Results_new.csv'

# Write the results DataFrame to the output CSV file
results_df.to_csv(output_csv_path, index=False)

# Display the results DataFrame
print("Results:")
print(results_df)


Processing file: 28353604_df.csv
Processing file: 28559815_df.csv
Processing file: 28538413_df.csv
Processing file: 28353588_df.csv
Processing file: 28353596_df.csv
Processing file: 28353613_df.csv
Processing file: 28353569_df.csv
Processing file: 28403092_df.csv
Processing file: 28403086_df.csv
Processing file: 28321071_df.csv
Processing file: 28353561_df.csv
Processing file: 28353556_df.csv
Processing file: 28353558_df.csv
Processing file: 28422883_df.csv
Processing file: 28383413_df.csv
Processing file: 28403099_df.csv
Processing file: 28321073_df.csv
Processing file: 28595573_df.csv
Processing file: 28767567_df.csv
Processing file: 28239141_df.csv
Processing file: 28216610_df.csv
Processing file: 28248858_df.csv
Processing file: 28202865_df.csv
Processing file: 28207542_df.csv
Processing file: 28196820_df.csv
Processing file: 28248891_df.csv
Processing file: 28202862_df.csv
Processing file: 28190872_df.csv
Processing file: 28193213_df.csv
Processing file: 28202869_df.csv
Processing